In [1]:
# imports
import pandas as pd
import agrilution_aws
import logging
import boto3
from datetime import datetime
import sys
from boto3.dynamodb.conditions import Key, Attr
import time
import ast
from agrilution_aws import DynamoDbApi
pd.options.mode.chained_assignment = None

In [5]:
dynamodb = boto3.resource('dynamodb')

table = dynamodb.Table('purchased_subscriptions_prod')

response = table.scan()

In [6]:
data = response['Items']

In [7]:
df = pd.DataFrame(data)

In [8]:
df.head(50)

,orders_created,created_at,subscription_id,address,_next_order_day,base_subscription,updated_at,is_first_order,is_cancelled,next_order_date,remaining_free_months,cancellable_from,subscription_code,user_id
0,0,2022-02-28T14:47:37.550471+01:00,843229,"{'country': 'DE', 'city': 'Hengersberg', 'stre...",1999-12-31,"{'subscription_group_billing_rules_id': 42057,...",2022-02-28T14:47:37.550471+01:00,True,False,1999-12-31T01:00:00+01:00,6,2499-01-01,60903415,NaN
1,0,2022-09-14T16:54:14.620877+02:00,1224245,"{'country': 'DE', 'city': 'Stuttgart', 'street...",1999-12-31,"{'subscription_group_billing_rules_id': 62993,...",2022-09-14T16:54:14.620877+02:00,True,False,1999-12-31T01:00:00+01:00,0,2499-01-01,NaN,eu-central-1:4d9ee6bf-13bd-4caa-8a10-002f225731d6
2,0,2022-05-27T23:10:25.043197+02:00,1017165,"{'country': 'DE', 'city': 'Hamburg', 'street':...",1999-12-31,"{'subscription_group_billing_rules_id': 42057,...",2022-05-27T23:10:25.043197+02:00,True,False,1999-12-31T01:00:00+01:00,6,2499-01-01,09875370,NaN
3,NaN,2022-02-07T21:46:13.662877+01:00,697714,"{'country': 'DE', 'city': 'Lüneburg', 'street'...",1999-12-31,"{'subscription_group_billing_rules_id': 42057,...",2022-02-07T21:46:13.662877+01:00,True,NaN,2000-01-01T00:00:00+01:00,6,2500-01-01,NaN,eu-central-1:ecb49cd2-006a-4796-b229-931f2bc2c964
4,1,2022-09-17T23:16:40.247219+02:00,1003852,"{'country': 'AT', 'city': 'Hagenberg im Mühlkr...",2022-11-11,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17T23:16:40.247219+02:00,False,False,2022-11-11T02:00:00+01:00,6,2024-09-08,NaN,eu-central-1:4c52ccd7-f8a0-467e-9817-2cfb0f32a591
5,0,2022-05-29T22:08:59.681135+02:00,1019288,"{'country': 'DE', 'city': 'Hamburg', 'street':...",1999-12-31,"{'subscription_group_billing_rules_id': 42057,...",2022-05-29T22:08:59.681135+02:00,True,False,1999-12-31T01:00:00+01:00,6,2499-01-01,54207488,NaN
6,1,2022-02-07T17:51:00.407136+01:00,703900,"{'country': 'DE', 'city': 'Kleinostheim', 'str...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17T01:01:46.479707+02:00,False,False,2022-11-17T02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:bb4ab762-8375-456c-b334-6f2246a591d0
7,4,2022-03-13T12:39:57.791882+01:00,866842,"{'country': 'DE', 'city': 'Bissendorf', 'stree...",2022-11-13,"{'subscription_group_billing_rules_id': 42054,...",2022-09-13T01:01:47.385277+02:00,False,False,2022-11-13T02:00:00+01:00,0,2023-03-13,NaN,eu-central-1:629f638d-8696-45bc-b07d-77adfde385f2
8,NaN,2022-02-12T11:54:50.378947+01:00,757389,"{'country': 'DE', 'city': 'Kloster Lehnin', 's...",1999-12-31,"{'subscription_group_billing_rules_id': 42057,...",2022-02-12T11:54:50.378947+01:00,True,NaN,1999-12-31T01:00:00+01:00,6,2499-01-01,31493731,NaN
9,0,2022-03-20T19:52:27.403404+01:00,884483,"{'country': 'DE', 'city': 'Alfeld (Leine)', 's...",1999-12-31,"{'subscription_group_billing_rules_id': 44996,...",2022-03-20T19:52:27.403404+01:00,True,False,1999-12-31T01:00:00+01:00,6,2499-01-01,38427267,NaN


In [9]:
df.columns

Index(['orders_created', 'created_at', 'subscription_id', 'address',
       '_next_order_day', 'base_subscription', 'updated_at', 'is_first_order',
       'is_cancelled', 'next_order_date', 'remaining_free_months',
       'cancellable_from', 'subscription_code', 'user_id'],
      dtype='object')

In [10]:
# PurchasedSubscription
# • subscription_id : Same as id from Bold.
# • BaseSubscription : Subscription’s info from Bold which will be mapped to a Shopify order.
# • next_order_date : When Bold created the order in Shopify.
# • _next_order_day : next_order_date but with precision up to the day for.
# • address : User address from Shopify.
# • cancellable_from : Date from which the subscription can be cancelled.
# • is_cancelled : Flag that indicates if a subscription is cancelled (more
# info
# ).
# • is_first_order : Flag to indicate whether the subscription has ever been processed by SeedsManagement lambda.
# • remaining_free_months : Number of free months remaiing for this subscription. Only applies to subscription bought through the
# configurator.
# • orders_created : number of orders created for this subscription.

In [11]:
df.loc[df['subscription_id'] == '1017165','base_subscription'].values

array([{'subscription_group_billing_rules_id': Decimal('42057'), 'price_charged': Decimal('0'), 'requires_shipping': True, 'prepaid_metadata': None, 'created_at': '2022-05-27T21:10:18Z', 'title': 'Premium Abo - 2 Jahre', 'platform_product_id': '6926102331583', 'discounted_price': Decimal('0'), 'subscription_id': Decimal('1017165'), 'discounts': [], 'updated_at': '2022-05-27T21:10:19Z', 'price': Decimal('0'), 'id': Decimal('1408548249'), 'sku': 'SUB2P#', 'grams': Decimal('0'), 'subscription_group_id': Decimal('20128'), 'full_price_charged': None, 'image': 'https://cdn.shopify.com/s/files/1/0597/9111/4431/products/App-Graphics_Help_Subscription.png?v=1641400295', 'quantity': Decimal('1'), 'taxable': True, 'platform_variant_id': '41102067630271', 'weight': Decimal('0'), 'variant_name': '44.99 € monatlich nach Plantcube Lieferung', 'product_name': 'Premium Abo - 2 Jahre', 'url': '/products/24-month-subscription-flat-for-6', 'weight_unit': 'kg', 'discounted_price_charged': Decimal('0'), 'fu

In [12]:
df.loc[df['subscription_id'] == '843229','base_subscription'].values

array([{'subscription_group_billing_rules_id': Decimal('42057'), 'price_charged': Decimal('0'), 'requires_shipping': True, 'prepaid_metadata': None, 'created_at': '2022-02-28T13:47:33Z', 'title': 'Premium Abo - 2 Jahre', 'platform_product_id': '6926102331583', 'discounted_price': Decimal('0'), 'subscription_id': Decimal('843229'), 'discounts': [], 'updated_at': '2022-02-28T13:47:34Z', 'price': Decimal('0'), 'id': Decimal('133438153'), 'sku': 'SUB2P#', 'grams': Decimal('0'), 'subscription_group_id': Decimal('20128'), 'full_price_charged': None, 'image': 'https://cdn.shopify.com/s/files/1/0597/9111/4431/products/App-Graphics_Help_Subscription.png?v=1641400295', 'quantity': Decimal('1'), 'taxable': True, 'platform_variant_id': '41102067630271', 'weight': Decimal('0'), 'variant_name': '44.99 € monatlich nach Plantcube Lieferung', 'product_name': 'Premium Abo - 2 Jahre', 'url': '/products/24-month-subscription-flat-for-6', 'weight_unit': 'kg', 'discounted_price_charged': Decimal('0'), 'full

In [13]:
df.loc[df['subscription_id'] == '1017165','address'].values

array([{'country': 'DE', 'city': 'Hamburg', 'street': 'Klopstockstraße 7', 'zip_code': '22765'}],
      dtype=object)

In [14]:
df.isnull().sum()

orders_created            69
created_at                44
subscription_id            0
address                    0
_next_order_day            0
base_subscription          0
updated_at                39
is_first_order             0
is_cancelled              69
next_order_date            0
remaining_free_months      0
cancellable_from           0
subscription_code        147
user_id                  135
dtype: int64

In [15]:
df['subscription_id'].nunique() 

282

In [16]:
df['_next_order_day'] = pd.to_datetime(df['_next_order_day'])

In [17]:
#If a subscription has not yet started, e.g. the _next_order_day is 1999-12-31
#Items in there are always updated, this table is not versioned. 
#once a subscription becomes active, _next_order_day becomes a date in the future like 2022-09-11

In [18]:
df1 = df[df['_next_order_day'].dt.year == 2022]

In [19]:
df1

,orders_created,created_at,subscription_id,address,_next_order_day,base_subscription,updated_at,is_first_order,is_cancelled,next_order_date,remaining_free_months,cancellable_from,subscription_code,user_id
4,1,2022-09-17T23:16:40.247219+02:00,1003852,"{'country': 'AT', 'city': 'Hagenberg im Mühlkr...",2022-11-11,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17T23:16:40.247219+02:00,False,False,2022-11-11T02:00:00+01:00,6,2024-09-08,NaN,eu-central-1:4c52ccd7-f8a0-467e-9817-2cfb0f32a591
6,1,2022-02-07T17:51:00.407136+01:00,703900,"{'country': 'DE', 'city': 'Kleinostheim', 'str...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17T01:01:46.479707+02:00,False,False,2022-11-17T02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:bb4ab762-8375-456c-b334-6f2246a591d0
7,4,2022-03-13T12:39:57.791882+01:00,866842,"{'country': 'DE', 'city': 'Bissendorf', 'stree...",2022-11-13,"{'subscription_group_billing_rules_id': 42054,...",2022-09-13T01:01:47.385277+02:00,False,False,2022-11-13T02:00:00+01:00,0,2023-03-13,NaN,eu-central-1:629f638d-8696-45bc-b07d-77adfde385f2
13,3,2022-05-31T10:06:10.960000+02:00,1021037,"{'country': 'CH', 'city': 'Champagne', 'street...",2022-11-04,"{'subscription_group_billing_rules_id': 44996,...",2022-09-04T01:01:52.620430+02:00,False,False,2022-11-04T02:00:00+01:00,0,2024-05-31,NaN,eu-central-1:9770b4df-de34-45de-89ff-596f9221a4dc
16,1,2022-09-06T18:28:41.960919+02:00,843024,"{'country': 'DE', 'city': 'Vilshofen an der Do...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17T01:01:56.280523+02:00,False,False,2022-11-17T02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:8a8dcdad-0224-4b04-bd1f-b9534bce89f1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,1,2022-09-12T12:39:46.657667+02:00,697760,"{'country': 'AT', 'city': 'Wien', 'street': 'F...",2022-11-09,"{'subscription_group_billing_rules_id': 44996,...",2022-09-12T12:39:46.657667+02:00,False,False,2022-11-09T02:00:00+01:00,6,2024-09-06,NaN,eu-central-1:fb18f5d8-751e-4945-815a-cf17c3c72edb
271,1,2022-08-27T10:19:29.628836+02:00,1003841,"{'country': 'DE', 'city': 'Hamburg', 'street':...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17T01:08:18.124691+02:00,False,False,2022-11-17T02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:3e076d12-c1a0-4a9d-b62b-90a1a081337d
273,1,2022-02-07T17:46:29.084959+01:00,697816,"{'country': 'DE', 'city': 'Ludwigsburg', 'stre...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17T01:08:27.163817+02:00,False,False,2022-11-17T02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:8ea796d5-6654-40d0-8285-dbeec929f07b
280,1,2022-08-03T18:46:27.296760+02:00,757336,"{'country': 'DE', 'city': 'Rheinberg', 'street...",2022-11-17,"{'subscription_group_billing_rules_id': 44996,...",2022-09-17T01:08:34.444570+02:00,False,False,2022-11-17T02:00:00+01:00,12,2024-09-14,NaN,eu-central-1:8b46d23f-a995-4a61-88e1-42948015a7c1


In [20]:
df1['updated_at'] = pd.to_datetime(df1['updated_at'])

In [21]:
df1['next_order_date'] = pd.to_datetime(df1['next_order_date'])

In [22]:
df1

,orders_created,created_at,subscription_id,address,_next_order_day,base_subscription,updated_at,is_first_order,is_cancelled,next_order_date,remaining_free_months,cancellable_from,subscription_code,user_id
4,1,2022-09-17T23:16:40.247219+02:00,1003852,"{'country': 'AT', 'city': 'Hagenberg im Mühlkr...",2022-11-11,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 23:16:40.247219+02:00,False,False,2022-11-11 02:00:00+01:00,6,2024-09-08,NaN,eu-central-1:4c52ccd7-f8a0-467e-9817-2cfb0f32a591
6,1,2022-02-07T17:51:00.407136+01:00,703900,"{'country': 'DE', 'city': 'Kleinostheim', 'str...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:01:46.479707+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:bb4ab762-8375-456c-b334-6f2246a591d0
7,4,2022-03-13T12:39:57.791882+01:00,866842,"{'country': 'DE', 'city': 'Bissendorf', 'stree...",2022-11-13,"{'subscription_group_billing_rules_id': 42054,...",2022-09-13 01:01:47.385277+02:00,False,False,2022-11-13 02:00:00+01:00,0,2023-03-13,NaN,eu-central-1:629f638d-8696-45bc-b07d-77adfde385f2
13,3,2022-05-31T10:06:10.960000+02:00,1021037,"{'country': 'CH', 'city': 'Champagne', 'street...",2022-11-04,"{'subscription_group_billing_rules_id': 44996,...",2022-09-04 01:01:52.620430+02:00,False,False,2022-11-04 02:00:00+01:00,0,2024-05-31,NaN,eu-central-1:9770b4df-de34-45de-89ff-596f9221a4dc
16,1,2022-09-06T18:28:41.960919+02:00,843024,"{'country': 'DE', 'city': 'Vilshofen an der Do...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:01:56.280523+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:8a8dcdad-0224-4b04-bd1f-b9534bce89f1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,1,2022-09-12T12:39:46.657667+02:00,697760,"{'country': 'AT', 'city': 'Wien', 'street': 'F...",2022-11-09,"{'subscription_group_billing_rules_id': 44996,...",2022-09-12 12:39:46.657667+02:00,False,False,2022-11-09 02:00:00+01:00,6,2024-09-06,NaN,eu-central-1:fb18f5d8-751e-4945-815a-cf17c3c72edb
271,1,2022-08-27T10:19:29.628836+02:00,1003841,"{'country': 'DE', 'city': 'Hamburg', 'street':...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:08:18.124691+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:3e076d12-c1a0-4a9d-b62b-90a1a081337d
273,1,2022-02-07T17:46:29.084959+01:00,697816,"{'country': 'DE', 'city': 'Ludwigsburg', 'stre...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:08:27.163817+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:8ea796d5-6654-40d0-8285-dbeec929f07b
280,1,2022-08-03T18:46:27.296760+02:00,757336,"{'country': 'DE', 'city': 'Rheinberg', 'street...",2022-11-17,"{'subscription_group_billing_rules_id': 44996,...",2022-09-17 01:08:34.444570+02:00,False,False,2022-11-17 02:00:00+01:00,12,2024-09-14,NaN,eu-central-1:8b46d23f-a995-4a61-88e1-42948015a7c1


In [23]:
df2 = df1.copy()

In [24]:
df2

,orders_created,created_at,subscription_id,address,_next_order_day,base_subscription,updated_at,is_first_order,is_cancelled,next_order_date,remaining_free_months,cancellable_from,subscription_code,user_id
4,1,2022-09-17T23:16:40.247219+02:00,1003852,"{'country': 'AT', 'city': 'Hagenberg im Mühlkr...",2022-11-11,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 23:16:40.247219+02:00,False,False,2022-11-11 02:00:00+01:00,6,2024-09-08,NaN,eu-central-1:4c52ccd7-f8a0-467e-9817-2cfb0f32a591
6,1,2022-02-07T17:51:00.407136+01:00,703900,"{'country': 'DE', 'city': 'Kleinostheim', 'str...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:01:46.479707+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:bb4ab762-8375-456c-b334-6f2246a591d0
7,4,2022-03-13T12:39:57.791882+01:00,866842,"{'country': 'DE', 'city': 'Bissendorf', 'stree...",2022-11-13,"{'subscription_group_billing_rules_id': 42054,...",2022-09-13 01:01:47.385277+02:00,False,False,2022-11-13 02:00:00+01:00,0,2023-03-13,NaN,eu-central-1:629f638d-8696-45bc-b07d-77adfde385f2
13,3,2022-05-31T10:06:10.960000+02:00,1021037,"{'country': 'CH', 'city': 'Champagne', 'street...",2022-11-04,"{'subscription_group_billing_rules_id': 44996,...",2022-09-04 01:01:52.620430+02:00,False,False,2022-11-04 02:00:00+01:00,0,2024-05-31,NaN,eu-central-1:9770b4df-de34-45de-89ff-596f9221a4dc
16,1,2022-09-06T18:28:41.960919+02:00,843024,"{'country': 'DE', 'city': 'Vilshofen an der Do...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:01:56.280523+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:8a8dcdad-0224-4b04-bd1f-b9534bce89f1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,1,2022-09-12T12:39:46.657667+02:00,697760,"{'country': 'AT', 'city': 'Wien', 'street': 'F...",2022-11-09,"{'subscription_group_billing_rules_id': 44996,...",2022-09-12 12:39:46.657667+02:00,False,False,2022-11-09 02:00:00+01:00,6,2024-09-06,NaN,eu-central-1:fb18f5d8-751e-4945-815a-cf17c3c72edb
271,1,2022-08-27T10:19:29.628836+02:00,1003841,"{'country': 'DE', 'city': 'Hamburg', 'street':...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:08:18.124691+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:3e076d12-c1a0-4a9d-b62b-90a1a081337d
273,1,2022-02-07T17:46:29.084959+01:00,697816,"{'country': 'DE', 'city': 'Ludwigsburg', 'stre...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:08:27.163817+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,NaN,eu-central-1:8ea796d5-6654-40d0-8285-dbeec929f07b
280,1,2022-08-03T18:46:27.296760+02:00,757336,"{'country': 'DE', 'city': 'Rheinberg', 'street...",2022-11-17,"{'subscription_group_billing_rules_id': 44996,...",2022-09-17 01:08:34.444570+02:00,False,False,2022-11-17 02:00:00+01:00,12,2024-09-14,NaN,eu-central-1:8b46d23f-a995-4a61-88e1-42948015a7c1


In [25]:
Address = df2.address.apply(pd.Series)

In [26]:
Base_Subscription = df2.base_subscription.apply(pd.Series)

In [27]:
Address.head()

,country,city,street,zip_code
4,AT,Hagenberg im Mühlkreis,Softwarepark 30,4232
6,DE,Kleinostheim,Im Wingert 33B,63801
7,DE,Bissendorf,Strügelheide 2,49143
13,CH,Champagne,Chemin de Pécheret 25,1424
16,DE,Vilshofen an der Donau,Alkofener Hauptstraße 105,94474


In [28]:
len(Address.index)

106

In [29]:
Base_Subscription.head()

,subscription_group_billing_rules_id,price_charged,requires_shipping,prepaid_metadata,created_at,title,platform_product_id,discounted_price,subscription_id,discounts,...,quantity,taxable,platform_variant_id,weight,variant_name,product_name,url,weight_unit,discounted_price_charged,full_price
4,42057,0,True,None,2022-05-21T09:12:49Z,Premium Abo - 2 Jahre,6926102331583,0,1003852,[],...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
6,42057,0,True,None,2022-01-15T12:21:05Z,Premium Abo - 2 Jahre,6926102331583,0,703900,[],...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
7,42054,6598,True,None,2022-03-13T11:39:51Z,Basic Abo - 1 Jahr,6912360382655,6598,866842,[],...,1,True,41002624745663,0,Paid,Basic Abo - 1 Jahr,/products/12-month-subscripton,kg,6598,None
13,44996,9800,True,None,2022-05-31T08:06:03Z,Basic Abo - 2 Jahre,6926045216959,9492,1021037,[],...,1,True,41102064550079,0,Paid,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,9800,None
16,42057,0,True,None,2022-02-28T09:23:58Z,Premium Abo - 2 Jahre,6926102331583,0,843024,[],...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None


In [30]:
len(Base_Subscription.index)

106

In [31]:
merged1 = Address.merge(Base_Subscription, left_index=True, right_index=True, how='inner',suffixes=('', '_Base'))

In [32]:
merged1

,country,city,street,zip_code,subscription_group_billing_rules_id,price_charged,requires_shipping,prepaid_metadata,created_at,title,...,quantity,taxable,platform_variant_id,weight,variant_name,product_name,url,weight_unit,discounted_price_charged,full_price
4,AT,Hagenberg im Mühlkreis,Softwarepark 30,4232,42057,0,True,None,2022-05-21T09:12:49Z,Premium Abo - 2 Jahre,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
6,DE,Kleinostheim,Im Wingert 33B,63801,42057,0,True,None,2022-01-15T12:21:05Z,Premium Abo - 2 Jahre,...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
7,DE,Bissendorf,Strügelheide 2,49143,42054,6598,True,None,2022-03-13T11:39:51Z,Basic Abo - 1 Jahr,...,1,True,41002624745663,0,Paid,Basic Abo - 1 Jahr,/products/12-month-subscripton,kg,6598,None
13,CH,Champagne,Chemin de Pécheret 25,1424,44996,9800,True,None,2022-05-31T08:06:03Z,Basic Abo - 2 Jahre,...,1,True,41102064550079,0,Paid,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,9800,None
16,DE,Vilshofen an der Donau,Alkofener Hauptstraße 105,94474,42057,0,True,None,2022-02-28T09:23:58Z,Premium Abo - 2 Jahre,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,AT,Wien,Flurschützstraße 1,1120,44996,0,True,None,2022-01-11T15:11:19Z,Basic Abo - 2 Jahre,...,1,True,41102064517311,0,29.99 € monatlich,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,0,None
271,DE,Hamburg,Isestraße 49,20144,42057,0,True,None,2022-05-21T08:44:43Z,Premium Abo - 2 Jahre,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
273,DE,Ludwigsburg,Gräterstrasse 10,71642,42057,0,True,None,2022-01-11T15:59:12Z,Premium Abo - 2 Jahre,...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
280,DE,Rheinberg,Eversaeler Strasse 33,47495,44996,0,True,None,2022-02-12T08:23:47Z,Basic Abo - 2 Jahre,...,1,True,41102064517311,0,29.99 € monatlich nach Plantcube Lieferung,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,0,None


In [33]:
merged1.columns

Index(['country', 'city', 'street', 'zip_code',
       'subscription_group_billing_rules_id', 'price_charged',
       'requires_shipping', 'prepaid_metadata', 'created_at', 'title',
       'platform_product_id', 'discounted_price', 'subscription_id',
       'discounts', 'updated_at', 'price', 'id', 'sku', 'grams',
       'subscription_group_id', 'full_price_charged', 'image', 'quantity',
       'taxable', 'platform_variant_id', 'weight', 'variant_name',
       'product_name', 'url', 'weight_unit', 'discounted_price_charged',
       'full_price'],
      dtype='object')

In [34]:
merged2 = df2.merge(merged1, left_index=True, right_index=True, how='inner',suffixes=('', '_Base'))

In [35]:
merged2

,orders_created,created_at,subscription_id,address,_next_order_day,base_subscription,updated_at,is_first_order,is_cancelled,next_order_date,...,quantity,taxable,platform_variant_id,weight,variant_name,product_name,url,weight_unit,discounted_price_charged,full_price
4,1,2022-09-17T23:16:40.247219+02:00,1003852,"{'country': 'AT', 'city': 'Hagenberg im Mühlkr...",2022-11-11,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 23:16:40.247219+02:00,False,False,2022-11-11 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
6,1,2022-02-07T17:51:00.407136+01:00,703900,"{'country': 'DE', 'city': 'Kleinostheim', 'str...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:01:46.479707+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
7,4,2022-03-13T12:39:57.791882+01:00,866842,"{'country': 'DE', 'city': 'Bissendorf', 'stree...",2022-11-13,"{'subscription_group_billing_rules_id': 42054,...",2022-09-13 01:01:47.385277+02:00,False,False,2022-11-13 02:00:00+01:00,...,1,True,41002624745663,0,Paid,Basic Abo - 1 Jahr,/products/12-month-subscripton,kg,6598,None
13,3,2022-05-31T10:06:10.960000+02:00,1021037,"{'country': 'CH', 'city': 'Champagne', 'street...",2022-11-04,"{'subscription_group_billing_rules_id': 44996,...",2022-09-04 01:01:52.620430+02:00,False,False,2022-11-04 02:00:00+01:00,...,1,True,41102064550079,0,Paid,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,9800,None
16,1,2022-09-06T18:28:41.960919+02:00,843024,"{'country': 'DE', 'city': 'Vilshofen an der Do...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:01:56.280523+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,1,2022-09-12T12:39:46.657667+02:00,697760,"{'country': 'AT', 'city': 'Wien', 'street': 'F...",2022-11-09,"{'subscription_group_billing_rules_id': 44996,...",2022-09-12 12:39:46.657667+02:00,False,False,2022-11-09 02:00:00+01:00,...,1,True,41102064517311,0,29.99 € monatlich,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,0,None
271,1,2022-08-27T10:19:29.628836+02:00,1003841,"{'country': 'DE', 'city': 'Hamburg', 'street':...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:08:18.124691+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
273,1,2022-02-07T17:46:29.084959+01:00,697816,"{'country': 'DE', 'city': 'Ludwigsburg', 'stre...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:08:27.163817+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
280,1,2022-08-03T18:46:27.296760+02:00,757336,"{'country': 'DE', 'city': 'Rheinberg', 'street...",2022-11-17,"{'subscription_group_billing_rules_id': 44996,...",2022-09-17 01:08:34.444570+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102064517311,0,29.99 € monatlich nach Plantcube Lieferung,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,0,None


In [36]:
merged2.columns

Index(['orders_created', 'created_at', 'subscription_id', 'address',
       '_next_order_day', 'base_subscription', 'updated_at', 'is_first_order',
       'is_cancelled', 'next_order_date', 'remaining_free_months',
       'cancellable_from', 'subscription_code', 'user_id', 'country', 'city',
       'street', 'zip_code', 'subscription_group_billing_rules_id',
       'price_charged', 'requires_shipping', 'prepaid_metadata',
       'created_at_Base', 'title', 'platform_product_id', 'discounted_price',
       'subscription_id_Base', 'discounts', 'updated_at_Base', 'price', 'id',
       'sku', 'grams', 'subscription_group_id', 'full_price_charged', 'image',
       'quantity', 'taxable', 'platform_variant_id', 'weight', 'variant_name',
       'product_name', 'url', 'weight_unit', 'discounted_price_charged',
       'full_price'],
      dtype='object')

In [37]:
merged2

,orders_created,created_at,subscription_id,address,_next_order_day,base_subscription,updated_at,is_first_order,is_cancelled,next_order_date,...,quantity,taxable,platform_variant_id,weight,variant_name,product_name,url,weight_unit,discounted_price_charged,full_price
4,1,2022-09-17T23:16:40.247219+02:00,1003852,"{'country': 'AT', 'city': 'Hagenberg im Mühlkr...",2022-11-11,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 23:16:40.247219+02:00,False,False,2022-11-11 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
6,1,2022-02-07T17:51:00.407136+01:00,703900,"{'country': 'DE', 'city': 'Kleinostheim', 'str...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:01:46.479707+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
7,4,2022-03-13T12:39:57.791882+01:00,866842,"{'country': 'DE', 'city': 'Bissendorf', 'stree...",2022-11-13,"{'subscription_group_billing_rules_id': 42054,...",2022-09-13 01:01:47.385277+02:00,False,False,2022-11-13 02:00:00+01:00,...,1,True,41002624745663,0,Paid,Basic Abo - 1 Jahr,/products/12-month-subscripton,kg,6598,None
13,3,2022-05-31T10:06:10.960000+02:00,1021037,"{'country': 'CH', 'city': 'Champagne', 'street...",2022-11-04,"{'subscription_group_billing_rules_id': 44996,...",2022-09-04 01:01:52.620430+02:00,False,False,2022-11-04 02:00:00+01:00,...,1,True,41102064550079,0,Paid,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,9800,None
16,1,2022-09-06T18:28:41.960919+02:00,843024,"{'country': 'DE', 'city': 'Vilshofen an der Do...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:01:56.280523+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,1,2022-09-12T12:39:46.657667+02:00,697760,"{'country': 'AT', 'city': 'Wien', 'street': 'F...",2022-11-09,"{'subscription_group_billing_rules_id': 44996,...",2022-09-12 12:39:46.657667+02:00,False,False,2022-11-09 02:00:00+01:00,...,1,True,41102064517311,0,29.99 € monatlich,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,0,None
271,1,2022-08-27T10:19:29.628836+02:00,1003841,"{'country': 'DE', 'city': 'Hamburg', 'street':...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:08:18.124691+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
273,1,2022-02-07T17:46:29.084959+01:00,697816,"{'country': 'DE', 'city': 'Ludwigsburg', 'stre...",2022-11-17,"{'subscription_group_billing_rules_id': 42057,...",2022-09-17 01:08:27.163817+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
280,1,2022-08-03T18:46:27.296760+02:00,757336,"{'country': 'DE', 'city': 'Rheinberg', 'street...",2022-11-17,"{'subscription_group_billing_rules_id': 44996,...",2022-09-17 01:08:34.444570+02:00,False,False,2022-11-17 02:00:00+01:00,...,1,True,41102064517311,0,29.99 € monatlich nach Plantcube Lieferung,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,0,None


In [38]:
merged2.drop('base_subscription', inplace=True, axis=1)

In [39]:
merged2.drop('address', inplace=True, axis=1)

In [40]:
merged2

,orders_created,created_at,subscription_id,_next_order_day,updated_at,is_first_order,is_cancelled,next_order_date,remaining_free_months,cancellable_from,...,quantity,taxable,platform_variant_id,weight,variant_name,product_name,url,weight_unit,discounted_price_charged,full_price
4,1,2022-09-17T23:16:40.247219+02:00,1003852,2022-11-11,2022-09-17 23:16:40.247219+02:00,False,False,2022-11-11 02:00:00+01:00,6,2024-09-08,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
6,1,2022-02-07T17:51:00.407136+01:00,703900,2022-11-17,2022-09-17 01:01:46.479707+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
7,4,2022-03-13T12:39:57.791882+01:00,866842,2022-11-13,2022-09-13 01:01:47.385277+02:00,False,False,2022-11-13 02:00:00+01:00,0,2023-03-13,...,1,True,41002624745663,0,Paid,Basic Abo - 1 Jahr,/products/12-month-subscripton,kg,6598,None
13,3,2022-05-31T10:06:10.960000+02:00,1021037,2022-11-04,2022-09-04 01:01:52.620430+02:00,False,False,2022-11-04 02:00:00+01:00,0,2024-05-31,...,1,True,41102064550079,0,Paid,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,9800,None
16,1,2022-09-06T18:28:41.960919+02:00,843024,2022-11-17,2022-09-17 01:01:56.280523+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,1,2022-09-12T12:39:46.657667+02:00,697760,2022-11-09,2022-09-12 12:39:46.657667+02:00,False,False,2022-11-09 02:00:00+01:00,6,2024-09-06,...,1,True,41102064517311,0,29.99 € monatlich,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,0,None
271,1,2022-08-27T10:19:29.628836+02:00,1003841,2022-11-17,2022-09-17 01:08:18.124691+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
273,1,2022-02-07T17:46:29.084959+01:00,697816,2022-11-17,2022-09-17 01:08:27.163817+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
280,1,2022-08-03T18:46:27.296760+02:00,757336,2022-11-17,2022-09-17 01:08:34.444570+02:00,False,False,2022-11-17 02:00:00+01:00,12,2024-09-14,...,1,True,41102064517311,0,29.99 € monatlich nach Plantcube Lieferung,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,0,None


In [41]:
merged2.columns

Index(['orders_created', 'created_at', 'subscription_id', '_next_order_day',
       'updated_at', 'is_first_order', 'is_cancelled', 'next_order_date',
       'remaining_free_months', 'cancellable_from', 'subscription_code',
       'user_id', 'country', 'city', 'street', 'zip_code',
       'subscription_group_billing_rules_id', 'price_charged',
       'requires_shipping', 'prepaid_metadata', 'created_at_Base', 'title',
       'platform_product_id', 'discounted_price', 'subscription_id_Base',
       'discounts', 'updated_at_Base', 'price', 'id', 'sku', 'grams',
       'subscription_group_id', 'full_price_charged', 'image', 'quantity',
       'taxable', 'platform_variant_id', 'weight', 'variant_name',
       'product_name', 'url', 'weight_unit', 'discounted_price_charged',
       'full_price'],
      dtype='object')

In [42]:
merged2.to_csv('purchased_subscriptions_prod.csv', index=False)

In [ ]:
#further processing

In [43]:
merged2.head()

,orders_created,created_at,subscription_id,_next_order_day,updated_at,is_first_order,is_cancelled,next_order_date,remaining_free_months,cancellable_from,...,quantity,taxable,platform_variant_id,weight,variant_name,product_name,url,weight_unit,discounted_price_charged,full_price
4,1,2022-09-17T23:16:40.247219+02:00,1003852,2022-11-11,2022-09-17 23:16:40.247219+02:00,False,False,2022-11-11 02:00:00+01:00,6,2024-09-08,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
6,1,2022-02-07T17:51:00.407136+01:00,703900,2022-11-17,2022-09-17 01:01:46.479707+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,...,1,True,41102067630271,0,44.99 € monatlich,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None
7,4,2022-03-13T12:39:57.791882+01:00,866842,2022-11-13,2022-09-13 01:01:47.385277+02:00,False,False,2022-11-13 02:00:00+01:00,0,2023-03-13,...,1,True,41002624745663,0,Paid,Basic Abo - 1 Jahr,/products/12-month-subscripton,kg,6598,None
13,3,2022-05-31T10:06:10.960000+02:00,1021037,2022-11-04,2022-09-04 01:01:52.620430+02:00,False,False,2022-11-04 02:00:00+01:00,0,2024-05-31,...,1,True,41102064550079,0,Paid,Basic Abo - 2 Jahre,/products/24-month-subscription-flat-for-4,kg,9800,None
16,1,2022-09-06T18:28:41.960919+02:00,843024,2022-11-17,2022-09-17 01:01:56.280523+02:00,False,False,2022-11-17 02:00:00+01:00,6,2024-09-14,...,1,True,41102067630271,0,44.99 € monatlich nach Plantcube Lieferung,Premium Abo - 2 Jahre,/products/24-month-subscription-flat-for-6,kg,0,None


In [44]:
merged2.columns

Index(['orders_created', 'created_at', 'subscription_id', '_next_order_day',
       'updated_at', 'is_first_order', 'is_cancelled', 'next_order_date',
       'remaining_free_months', 'cancellable_from', 'subscription_code',
       'user_id', 'country', 'city', 'street', 'zip_code',
       'subscription_group_billing_rules_id', 'price_charged',
       'requires_shipping', 'prepaid_metadata', 'created_at_Base', 'title',
       'platform_product_id', 'discounted_price', 'subscription_id_Base',
       'discounts', 'updated_at_Base', 'price', 'id', 'sku', 'grams',
       'subscription_group_id', 'full_price_charged', 'image', 'quantity',
       'taxable', 'platform_variant_id', 'weight', 'variant_name',
       'product_name', 'url', 'weight_unit', 'discounted_price_charged',
       'full_price'],
      dtype='object')

In [102]:
cols = [11,2,29,21,34,2,5,20,26,9,5,3,6]
df1 = merged2[merged2.columns[cols]]

In [103]:
df1

,user_id,subscription_id,sku,title,quantity,subscription_id,is_first_order,created_at_Base,updated_at_Base,cancellable_from,is_first_order,_next_order_day,is_cancelled
4,eu-central-1:4c52ccd7-f8a0-467e-9817-2cfb0f32a591,1003852,SUB2P#,Premium Abo - 2 Jahre,1,1003852,False,2022-05-21T09:12:49Z,2022-05-21T09:12:50Z,2024-09-08,False,2022-11-11,False
6,eu-central-1:bb4ab762-8375-456c-b334-6f2246a591d0,703900,SUB2P#,Premium Abo - 2 Jahre,1,703900,False,2022-01-15T12:21:05Z,2022-01-15T12:21:06Z,2024-09-14,False,2022-11-17,False
7,eu-central-1:629f638d-8696-45bc-b07d-77adfde385f2,866842,SUB1B,Basic Abo - 1 Jahr,1,866842,False,2022-03-13T11:39:51Z,2022-03-13T11:39:52Z,2023-03-13,False,2022-11-13,False
13,eu-central-1:9770b4df-de34-45de-89ff-596f9221a4dc,1021037,SUB2B,Basic Abo - 2 Jahre,1,1021037,False,2022-05-31T08:06:03Z,2022-05-31T08:06:04Z,2024-05-31,False,2022-11-04,False
16,eu-central-1:8a8dcdad-0224-4b04-bd1f-b9534bce89f1,843024,SUB2P#,Premium Abo - 2 Jahre,1,843024,False,2022-02-28T09:23:58Z,2022-02-28T09:23:58Z,2024-09-14,False,2022-11-17,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,eu-central-1:fb18f5d8-751e-4945-815a-cf17c3c72edb,697760,SUB2B#,Basic Abo - 2 Jahre,1,697760,False,2022-01-11T15:11:19Z,2022-01-11T15:11:20Z,2024-09-06,False,2022-11-09,False
271,eu-central-1:3e076d12-c1a0-4a9d-b62b-90a1a081337d,1003841,SUB2P#,Premium Abo - 2 Jahre,1,1003841,False,2022-05-21T08:44:43Z,2022-05-21T08:44:44Z,2024-09-14,False,2022-11-17,False
273,eu-central-1:8ea796d5-6654-40d0-8285-dbeec929f07b,697816,SUB2P#,Premium Abo - 2 Jahre,1,697816,False,2022-01-11T15:59:12Z,2022-01-11T15:59:12Z,2024-09-14,False,2022-11-17,False
280,eu-central-1:8b46d23f-a995-4a61-88e1-42948015a7c1,757336,SUB2B#,Basic Abo - 2 Jahre,1,757336,False,2022-02-12T08:23:47Z,2022-02-12T08:23:48Z,2024-09-14,False,2022-11-17,False


In [104]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 4 to 281
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           94 non-null     object        
 1   subscription_id   106 non-null    object        
 2   sku               106 non-null    object        
 3   title             106 non-null    object        
 4   quantity          106 non-null    object        
 5   subscription_id   106 non-null    object        
 6   is_first_order    106 non-null    bool          
 7   created_at_Base   106 non-null    object        
 8   updated_at_Base   106 non-null    object        
 9   cancellable_from  106 non-null    object        
 10  is_first_order    106 non-null    bool          
 11  _next_order_day   106 non-null    datetime64[ns]
 12  is_cancelled      106 non-null    object        
dtypes: bool(2), datetime64[ns](1), object(10)
memory usage: 14.2+ KB


In [105]:
df1 = df1.astype({"is_cancelled": bool})

df1['created_at_Base'] = pd.to_datetime(df1['created_at_Base']).dt.strftime('%m/%d/%Y')
df1['updated_at_Base'] = pd.to_datetime(df1['updated_at_Base']).dt.strftime('%m/%d/%Y')
df1['cancellable_from'] = pd.to_datetime(df1['cancellable_from']).dt.strftime('%m/%d/%Y')

In [106]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 4 to 281
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           94 non-null     object        
 1   subscription_id   106 non-null    object        
 2   sku               106 non-null    object        
 3   title             106 non-null    object        
 4   quantity          106 non-null    object        
 5   subscription_id   106 non-null    object        
 6   is_first_order    106 non-null    bool          
 7   created_at_Base   106 non-null    object        
 8   updated_at_Base   106 non-null    object        
 9   cancellable_from  106 non-null    object        
 10  is_first_order    106 non-null    bool          
 11  _next_order_day   106 non-null    datetime64[ns]
 12  is_cancelled      106 non-null    bool          
dtypes: bool(3), datetime64[ns](1), object(9)
memory usage: 13.5+ KB


In [107]:
df1.head()

,user_id,subscription_id,sku,title,quantity,subscription_id,is_first_order,created_at_Base,updated_at_Base,cancellable_from,is_first_order,_next_order_day,is_cancelled
4,eu-central-1:4c52ccd7-f8a0-467e-9817-2cfb0f32a591,1003852,SUB2P#,Premium Abo - 2 Jahre,1,1003852,False,05/21/2022,05/21/2022,09/08/2024,False,2022-11-11,False
6,eu-central-1:bb4ab762-8375-456c-b334-6f2246a591d0,703900,SUB2P#,Premium Abo - 2 Jahre,1,703900,False,01/15/2022,01/15/2022,09/14/2024,False,2022-11-17,False
7,eu-central-1:629f638d-8696-45bc-b07d-77adfde385f2,866842,SUB1B,Basic Abo - 1 Jahr,1,866842,False,03/13/2022,03/13/2022,03/13/2023,False,2022-11-13,False
13,eu-central-1:9770b4df-de34-45de-89ff-596f9221a4dc,1021037,SUB2B,Basic Abo - 2 Jahre,1,1021037,False,05/31/2022,05/31/2022,05/31/2024,False,2022-11-04,False
16,eu-central-1:8a8dcdad-0224-4b04-bd1f-b9534bce89f1,843024,SUB2P#,Premium Abo - 2 Jahre,1,843024,False,02/28/2022,02/28/2022,09/14/2024,False,2022-11-17,False


In [108]:
df1['created_at_Base'] = pd.to_datetime(df1['created_at_Base'])
df1['updated_at_Base'] = pd.to_datetime(df1['updated_at_Base'])
df1['cancellable_from'] = pd.to_datetime(df1['cancellable_from'])
df1['Remanining Days'] = df1.cancellable_from - df1.updated_at_Base
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 4 to 281
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   user_id           94 non-null     object         
 1   subscription_id   106 non-null    object         
 2   sku               106 non-null    object         
 3   title             106 non-null    object         
 4   quantity          106 non-null    object         
 5   subscription_id   106 non-null    object         
 6   is_first_order    106 non-null    bool           
 7   created_at_Base   106 non-null    datetime64[ns] 
 8   updated_at_Base   106 non-null    datetime64[ns] 
 9   cancellable_from  106 non-null    datetime64[ns] 
 10  is_first_order    106 non-null    bool           
 11  _next_order_day   106 non-null    datetime64[ns] 
 12  is_cancelled      106 non-null    bool           
 13  Remanining Days   106 non-null    timedelta64[ns]
dtypes: bool(3)

In [109]:
df1.head()

,user_id,subscription_id,sku,title,quantity,subscription_id,is_first_order,created_at_Base,updated_at_Base,cancellable_from,is_first_order,_next_order_day,is_cancelled,Remanining Days
4,eu-central-1:4c52ccd7-f8a0-467e-9817-2cfb0f32a591,1003852,SUB2P#,Premium Abo - 2 Jahre,1,1003852,False,2022-05-21,2022-05-21,2024-09-08,False,2022-11-11,False,841 days
6,eu-central-1:bb4ab762-8375-456c-b334-6f2246a591d0,703900,SUB2P#,Premium Abo - 2 Jahre,1,703900,False,2022-01-15,2022-01-15,2024-09-14,False,2022-11-17,False,973 days
7,eu-central-1:629f638d-8696-45bc-b07d-77adfde385f2,866842,SUB1B,Basic Abo - 1 Jahr,1,866842,False,2022-03-13,2022-03-13,2023-03-13,False,2022-11-13,False,365 days
13,eu-central-1:9770b4df-de34-45de-89ff-596f9221a4dc,1021037,SUB2B,Basic Abo - 2 Jahre,1,1021037,False,2022-05-31,2022-05-31,2024-05-31,False,2022-11-04,False,731 days
16,eu-central-1:8a8dcdad-0224-4b04-bd1f-b9534bce89f1,843024,SUB2P#,Premium Abo - 2 Jahre,1,843024,False,2022-02-28,2022-02-28,2024-09-14,False,2022-11-17,False,929 days
